In [ ]:
import pandas as pd
import networkx as nx
import matplotlib
import matplotlib.pyplot as plt
import time
from itertools import chain
import numpy as np
from numpy import trapz
import seaborn as sns
import statsmodels.formula.api as smf
from scipy import stats
matplotlib.style.use('fivethirtyeight')

# Read in Data and Set Up DataFrames

## Demographic Data

In [ ]:
Demographic info
#Get variable for white/nonwhite
demodf = pd.read_csv('pdem02.csv')
relcols =['subjectkey','demo_race_a_p___10', 'demo_race_a_p___11', 'demo_ethn_v2', 'demo_race_a_p___18', 'demo_race_a_p___19', 'demo_race_a_p___20', 'demo_race_a_p___21',
         'demo_race_a_p___22', 'demo_race_a_p___23', 'demo_race_a_p___24', 'demo_comb_income_v2']
demodf = demodf[relcols].copy()

demodf.set_index('subjectkey', inplace=True)

demodf['Asian'] = demodf[['demo_race_a_p___18', 'demo_race_a_p___19', 'demo_race_a_p___20', 'demo_race_a_p___21', 'demo_race_a_p___22', 'demo_race_a_p___23', 'demo_race_a_p___24']].any(axis='columns')
demodf['White'] = demodf['demo_race_a_p___10']
demodf['Black'] =demodf['demo_race_a_p___11']
demodf['Latinx'] = demodf['demo_ethn_v2']

#Recode latinx 
recodedict= {1:1, 2:0}
demodf['Latinx']=demodf['Latinx'].replace(recodedict)

In [ ]:
#Income 

recode25 = {1: 1, 2:1, 3:1, 4:1, 5:0, 6:0, 7:0, 8:0, 9:0, 10:0, 777:0, 999:0}
recode50 = {1: 0, 2:0, 3:0, 4:0, 5:1, 6:1, 7:0, 8:0, 9:0, 10:0, 777:0, 999:0}
recode100 = {1: 0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:1, 8:1, 9:0,10:0, 777:0, 999:0}
recode200 = {1: 0, 2:0, 3:0, 4:0, 5:0, 6:0, 7:0, 8:0, 9:1, 10:1, 777:0, 999:0}

demodf['<$25k'] = demodf['demo_comb_income_v2']
demodf['<$25k'] = demodf['<$25k'].replace(recode25)
demodf['$25-50k']= demodf['demo_comb_income_v2']
demodf['$25-50k'] = demodf['$25-50k'].replace(recode50)
demodf['$50-100k'] = demodf['demo_comb_income_v2']
demodf['$50-100k'] = demodf['$50-100k'].replace(recode100)
demodf['>$100k'] = demodf['demo_comb_income_v2']
demodf['>$100k'] = demodf['>$100k'].replace(recode200)

## Participant Dataframe

In [ ]:
df = pd.read_csv('participants.csv')

df.set_index('subjectkey', inplace=True)

In [ ]:
#delete columns that are only zeros (i.e. columns that are only included in one timepoint)
participants = df.loc[:, (df != 0).any(axis=0)]

## Functional Connectivity

In [ ]:
restqc= pd.read_csv('abcd_betnet02.csv')
restqc= restqc[['subjectkey', 'eventname', 'rsfmri_c_ngd_ntpoints']].copy()
restqc.set_index('subjectkey', inplace=True)
display(restqc)

In [ ]:
#set up timepoint dfs
restqct1 = restqc[restqc['eventname']== 'baseline_year_1_arm_1']
restqct2 = restqc[restqc['eventname']== '2_year_follow_up_y_arm_1']

#timepoint1
restqct1 = restqct1.rename(columns= {'eventname':'eventname', 'rsfmri_c_ngd_ntpoints':'fmri_tpoint1'})
restqct1= restqct1[['fmri_tpoint1']].copy()

#timepoint2
restqct2 = restqct2.rename(columns= {'eventname':'eventname', 'rsfmri_c_ngd_ntpoints':'fmri_tpoint2'})
restqct2= restqct2[['fmri_tpoint2']].copy()

## Structural Measures

In [ ]:
# apply exclusion criteria 
qc = pd.read_csv('freesqc01.csv')
qc = qc[['subjectkey','eventname','fsqc_qc']].copy()

In [ ]:
#timepoint dfs:
qct1 = qc[qc['eventname']== 'baseline_year_1_arm_1']
qct2 = qc[qc['eventname']== '2_year_follow_up_y_arm_1']

qct1 = qct1[['subjectkey', 'fsqc_qc', 'iqc_rsfmri_ok_ser','iqc_t1_ok_ser','iqc_rsfmri_ok_sub_02_filt_nody']]
qct2 = qct2[['subjectkey', 'fsqc_qc', 'iqc_rsfmri_ok_ser','iqc_t1_ok_ser','iqc_rsfmri_ok_sub_02_filt_nody']]

qct1.set_index('subjectkey', inplace=True)
qct2.set_index('subjectkey', inplace=True)

## Merge Structural and Functional Measures

In [ ]:
time1mri = qct1.merge(restqct1, left_index=True, right_index=True)

In [ ]:
time2mri = qct2.merge(restqct2, left_index=True, right_index=True)

## Inclusion

In [ ]:
def inclusion(row, timepoint):
    if row['fsqc_qc'] == 1:
        if row[timepoint]>375:
            if row['iqc_rsfmri_ok_ser'] >2:
                if row['iqc_t1_ok_ser'] != 0:
                    if row['iqc_rsfmri_ok_sub_02_filt_nody'] > 600:
                        return 1
                    else:
                        return 0
                else:
                    return 0
            else:
                return 0
        else:
            return 0
    else:
        return 0
    

In [ ]:
time1mri['pass'] = time1mri.apply(lambda row: inclusion(row, 'fmri_tpoint1'),axis=1)
time2mri['pass'] = time2mri.apply(
    row: inclusion(row, 'fmri_tpoint2'),axis=1)

In [ ]:
mripass = time1mri.merge(time2mri, left_index=True, right_index=True)

In [ ]:
participants2 = participants.merge(mripass, left_index=True, right_index=True)

## Set up full participant dataframe

In [ ]:
weekdayst = ['screen1_wkdy_y.t0', 'screen2_wkdy_y.t0', 'screen3_wkdy_y.t0', 'screen4_wkdy_y.t0', 'screen5_wkdy_y.t0', 'screen_wkdy_y.t0']

weekendst = ['screen7_wknd_y.t0', 'screen8_wknd_y.t0', 'screen9_wknd_y.t0', 'screen10_wknd_y.t0', 'screen11_wknd_y.t0', 'screen12_wknd_y.t0']

In [ ]:
wkdystsum = participants[weekdayst].sum(axis=1)
wkndstsum = participants[weekendst].sum(axis=1)

In [ ]:
ST = 5*wkdystsum + 2*wkndstsum
participants['ST.t0'] = ST

In [ ]:
#this is legacy code from when structural and functional code were separated - need validp for later code
validp = participants

## Set up subcortical averages

In [ ]:
#establish the columns to be averaged
aud_to_sub = ['rsfmri_cor_ngd_au_scs_crcxlh', 'rsfmri_cor_ngd_au_scs_thplh', 'rsfmri_cor_ngd_au_scs_cdelh', 'rsfmri_cor_ngd_au_scs_ptlh', 'rsfmri_cor_ngd_au_scs_pllh', 'rsfmri_cor_ngd_au_scs_bs', 'rsfmri_cor_ngd_au_scs_hplh', 'rsfmri_cor_ngd_au_scs_aglh', 'rsfmri_cor_ngd_au_scs_aalh', 'rsfmri_cor_ngd_au_scs_vtdclh', 'rsfmri_cor_ngd_au_scs_crcxrh', 'rsfmri_cor_ngd_au_scs_thprh', 'rsfmri_cor_ngd_au_scs_cderh', 'rsfmri_cor_ngd_au_scs_ptrh', 'rsfmri_cor_ngd_au_scs_plrh', 'rsfmri_cor_ngd_au_scs_hprh', 'rsfmri_cor_ngd_au_scs_agrh', 'rsfmri_cor_ngd_au_scs_aarh', 'rsfmri_cor_ngd_au_scs_vtdcrh']

cingoper_to_sub = ['rsfmri_cor_ngd_cerc_scs_crcxlh', 'rsfmri_cor_ngd_cerc_scs_thplh', 'rsfmri_cor_ngd_cerc_scs_cdelh', 'rsfmri_cor_ngd_cerc_scs_ptlh', 'rsfmri_cor_ngd_cerc_scs_pllh', 'rsfmri_cor_ngd_cerc_scs_bs', 'rsfmri_cor_ngd_cerc_scs_hplh', 'rsfmri_cor_ngd_cerc_scs_aglh', 'rsfmri_cor_ngd_cerc_scs_aalh', 'rsfmri_cor_ngd_cerc_scs_vtdclh', 'rsfmri_cor_ngd_cerc_scs_crcxrh', 'rsfmri_cor_ngd_cerc_scs_thprh', 'rsfmri_cor_ngd_cerc_scs_cderh', 'rsfmri_cor_ngd_cerc_scs_ptrh', 'rsfmri_cor_ngd_cerc_scs_plrh', 'rsfmri_cor_ngd_cerc_scs_hprh', 'rsfmri_cor_ngd_cerc_scs_agrh', 'rsfmri_cor_ngd_cerc_scs_aarh', 'rsfmri_cor_ngd_cerc_scs_vtdcrh']

cingpar_to_sub = ['rsfmri_cor_ngd_copa_scs_crcxlh', 'rsfmri_cor_ngd_copa_scs_thplh', 'rsfmri_cor_ngd_copa_scs_cdelh', 'rsfmri_cor_ngd_copa_scs_ptlh', 'rsfmri_cor_ngd_copa_scs_pllh', 'rsfmri_cor_ngd_copa_scs_bs', 'rsfmri_cor_ngd_copa_scs_hplh', 'rsfmri_cor_ngd_copa_scs_aglh', 'rsfmri_cor_ngd_copa_scs_aalh', 'rsfmri_cor_ngd_copa_scs_vtdclh', 'rsfmri_cor_ngd_copa_scs_crcxrh', 'rsfmri_cor_ngd_copa_scs_thprh', 'rsfmri_cor_ngd_copa_scs_cderh', 'rsfmri_cor_ngd_copa_scs_ptrh', 'rsfmri_cor_ngd_copa_scs_plrh', 'rsfmri_cor_ngd_copa_scs_hprh', 'rsfmri_cor_ngd_copa_scs_agrh', 'rsfmri_cor_ngd_copa_scs_aarh', 'rsfmri_cor_ngd_copa_scs_vtdcrh']

default_to_sub = ['rsfmri_cor_ngd_df_scs_crcxlh', 'rsfmri_cor_ngd_df_scs_thplh', 'rsfmri_cor_ngd_df_scs_cdelh', 'rsfmri_cor_ngd_df_scs_ptlh', 'rsfmri_cor_ngd_df_scs_pllh', 'rsfmri_cor_ngd_df_scs_bs', 'rsfmri_cor_ngd_df_scs_hplh', 'rsfmri_cor_ngd_df_scs_aglh', 'rsfmri_cor_ngd_df_scs_aalh', 'rsfmri_cor_ngd_df_scs_vtdclh', 'rsfmri_cor_ngd_df_scs_crcxrh', 'rsfmri_cor_ngd_df_scs_thprh', 'rsfmri_cor_ngd_df_scs_cderh', 'rsfmri_cor_ngd_df_scs_ptrh', 'rsfmri_cor_ngd_df_scs_plrh', 'rsfmri_cor_ngd_df_scs_hprh', 'rsfmri_cor_ngd_df_scs_agrh', 'rsfmri_cor_ngd_df_scs_aarh', 'rsfmri_cor_ngd_df_scs_vtdcrh']

DAN_to_sub = ['rsfmri_cor_ngd_dsa_scs_crcxlh', 'rsfmri_cor_ngd_dsa_scs_thplh', 'rsfmri_cor_ngd_dsa_scs_cdelh', 'rsfmri_cor_ngd_dsa_scs_ptlh', 'rsfmri_cor_ngd_dsa_scs_pllh', 'rsfmri_cor_ngd_dsa_scs_bs', 'rsfmri_cor_ngd_dsa_scs_hplh', 'rsfmri_cor_ngd_dsa_scs_aglh', 'rsfmri_cor_ngd_dsa_scs_aalh', 'rsfmri_cor_ngd_dsa_scs_vtdclh', 'rsfmri_cor_ngd_dsa_scs_crcxrh', 'rsfmri_cor_ngd_dsa_scs_thprh', 'rsfmri_cor_ngd_dsa_scs_cderh', 'rsfmri_cor_ngd_dsa_scs_ptrh', 'rsfmri_cor_ngd_dsa_scs_plrh', 'rsfmri_cor_ngd_dsa_scs_hprh', 'rsfmri_cor_ngd_dsa_scs_agrh', 'rsfmri_cor_ngd_dsa_scs_aarh', 'rsfmri_cor_ngd_dsa_scs_vtdcrh']

frontopar_to_sub = ['rsfmri_cor_ngd_fopa_scs_crcxlh', 'rsfmri_cor_ngd_fopa_scs_thplh', 'rsfmri_cor_ngd_fopa_scs_cdelh', 'rsfmri_cor_ngd_fopa_scs_ptlh', 'rsfmri_cor_ngd_fopa_scs_pllh', 'rsfmri_cor_ngd_fopa_scs_bs', 'rsfmri_cor_ngd_fopa_scs_hplh', 'rsfmri_cor_ngd_fopa_scs_aglh', 'rsfmri_cor_ngd_fopa_scs_aalh', 'rsfmri_cor_ngd_fopa_scs_vtdclh', 'rsfmri_cor_ngd_fopa_scs_crcxrh', 'rsfmri_cor_ngd_fopa_scs_thprh', 'rsfmri_cor_ngd_fopa_scs_cderh', 'rsfmri_cor_ngd_fopa_scs_ptrh', 'rsfmri_cor_ngd_fopa_scs_plrh', 'rsfmri_cor_ngd_fopa_scs_hprh', 'rsfmri_cor_ngd_fopa_scs_agrh', 'rsfmri_cor_ngd_fopa_scs_aarh', 'rsfmri_cor_ngd_fopa_scs_vtdcrh']

other_to_sub = ['rsfmri_cor_ngd_none_scs_crcxlh', 'rsfmri_cor_ngd_none_scs_thplh', 'rsfmri_cor_ngd_none_scs_cdelh', 'rsfmri_cor_ngd_none_scs_ptlh', 'rsfmri_cor_ngd_none_scs_pllh', 'rsfmri_cor_ngd_none_scs_bs', 'rsfmri_cor_ngd_none_scs_hplh', 'rsfmri_cor_ngd_none_scs_aglh', 'rsfmri_cor_ngd_none_scs_aalh', 'rsfmri_cor_ngd_none_scs_vtdclh', 'rsfmri_cor_ngd_none_scs_crcxrh', 'rsfmri_cor_ngd_none_scs_thprh', 'rsfmri_cor_ngd_none_scs_cderh', 'rsfmri_cor_ngd_none_scs_ptrh', 'rsfmri_cor_ngd_none_scs_plrh', 'rsfmri_cor_ngd_none_scs_hprh', 'rsfmri_cor_ngd_none_scs_agrh', 'rsfmri_cor_ngd_none_scs_aarh', 'rsfmri_cor_ngd_none_scs_vtdcrh']

retrotemp_to_sub =['rsfmri_cor_ngd_rst_scs_crcxlh', 'rsfmri_cor_ngd_rst_scs_thplh', 'rsfmri_cor_ngd_rst_scs_cdelh', 'rsfmri_cor_ngd_rst_scs_ptlh', 'rsfmri_cor_ngd_rst_scs_pllh', 'rsfmri_cor_ngd_rst_scs_bs', 'rsfmri_cor_ngd_rst_scs_hplh', 'rsfmri_cor_ngd_rst_scs_aglh', 'rsfmri_cor_ngd_rst_scs_aalh', 'rsfmri_cor_ngd_rst_scs_vtdclh', 'rsfmri_cor_ngd_rst_scs_crcxrh', 'rsfmri_cor_ngd_rst_scs_thprh', 'rsfmri_cor_ngd_rst_scs_cderh', 'rsfmri_cor_ngd_rst_scs_ptrh', 'rsfmri_cor_ngd_rst_scs_plrh', 'rsfmri_cor_ngd_rst_scs_hprh', 'rsfmri_cor_ngd_rst_scs_agrh', 'rsfmri_cor_ngd_rst_scs_aarh', 'rsfmri_cor_ngd_rst_scs_vtdcrh']

senshand_to_sub = ['rsfmri_cor_ngd_smh_scs_crcxlh', 'rsfmri_cor_ngd_smh_scs_thplh', 'rsfmri_cor_ngd_smh_scs_cdelh', 'rsfmri_cor_ngd_smh_scs_ptlh', 'rsfmri_cor_ngd_smh_scs_pllh', 'rsfmri_cor_ngd_smh_scs_bs', 'rsfmri_cor_ngd_smh_scs_hplh', 'rsfmri_cor_ngd_smh_scs_aglh', 'rsfmri_cor_ngd_smh_scs_aalh', 'rsfmri_cor_ngd_smh_scs_vtdclh', 'rsfmri_cor_ngd_smh_scs_crcxrh', 'rsfmri_cor_ngd_smh_scs_thprh', 'rsfmri_cor_ngd_smh_scs_cderh', 'rsfmri_cor_ngd_smh_scs_ptrh', 'rsfmri_cor_ngd_smh_scs_plrh', 'rsfmri_cor_ngd_smh_scs_hprh', 'rsfmri_cor_ngd_smh_scs_agrh', 'rsfmri_cor_ngd_smh_scs_aarh', 'rsfmri_cor_ngd_smh_scs_vtdcrh']

sensmouth_to_sub = ['rsfmri_cor_ngd_smm_scs_crcxlh', 'rsfmri_cor_ngd_smm_scs_thplh', 'rsfmri_cor_ngd_smm_scs_cdelh', 'rsfmri_cor_ngd_smm_scs_ptlh', 'rsfmri_cor_ngd_smm_scs_pllh', 'rsfmri_cor_ngd_smm_scs_bs', 'rsfmri_cor_ngd_smm_scs_hplh', 'rsfmri_cor_ngd_smm_scs_aglh', 'rsfmri_cor_ngd_smm_scs_aalh', 'rsfmri_cor_ngd_smm_scs_vtdclh', 'rsfmri_cor_ngd_smm_scs_crcxrh', 'rsfmri_cor_ngd_smm_scs_thprh', 'rsfmri_cor_ngd_smm_scs_cderh', 'rsfmri_cor_ngd_smm_scs_ptrh', 'rsfmri_cor_ngd_smm_scs_plrh', 'rsfmri_cor_ngd_smm_scs_hprh', 'rsfmri_cor_ngd_smm_scs_agrh', 'rsfmri_cor_ngd_smm_scs_aarh', 'rsfmri_cor_ngd_smm_scs_vtdcrh']

salience_to_sub = ['rsfmri_cor_ngd_sa_scs_crcxlh', 'rsfmri_cor_ngd_sa_scs_thplh', 'rsfmri_cor_ngd_sa_scs_cdelh', 'rsfmri_cor_ngd_sa_scs_ptlh', 'rsfmri_cor_ngd_sa_scs_pllh', 'rsfmri_cor_ngd_sa_scs_bs', 'rsfmri_cor_ngd_sa_scs_hplh', 'rsfmri_cor_ngd_sa_scs_aglh', 'rsfmri_cor_ngd_sa_scs_aalh', 'rsfmri_cor_ngd_sa_scs_vtdclh', 'rsfmri_cor_ngd_sa_scs_crcxrh', 'rsfmri_cor_ngd_sa_scs_thprh', 'rsfmri_cor_ngd_sa_scs_cderh', 'rsfmri_cor_ngd_sa_scs_ptrh', 'rsfmri_cor_ngd_sa_scs_plrh', 'rsfmri_cor_ngd_sa_scs_hprh', 'rsfmri_cor_ngd_sa_scs_agrh', 'rsfmri_cor_ngd_sa_scs_aarh', 'rsfmri_cor_ngd_sa_scs_vtdcrh']

VAN_to_sub = ['rsfmri_cor_ngd_vta_scs_crcxlh', 'rsfmri_cor_ngd_vta_scs_thplh', 'rsfmri_cor_ngd_vta_scs_cdelh', 'rsfmri_cor_ngd_vta_scs_ptlh', 'rsfmri_cor_ngd_vta_scs_pllh', 'rsfmri_cor_ngd_vta_scs_bs', 'rsfmri_cor_ngd_vta_scs_hplh', 'rsfmri_cor_ngd_vta_scs_aglh', 'rsfmri_cor_ngd_vta_scs_aalh', 'rsfmri_cor_ngd_vta_scs_vtdclh', 'rsfmri_cor_ngd_vta_scs_crcxrh', 'rsfmri_cor_ngd_vta_scs_thprh', 'rsfmri_cor_ngd_vta_scs_cderh', 'rsfmri_cor_ngd_vta_scs_ptrh', 'rsfmri_cor_ngd_vta_scs_plrh', 'rsfmri_cor_ngd_vta_scs_hprh', 'rsfmri_cor_ngd_vta_scs_agrh', 'rsfmri_cor_ngd_vta_scs_aarh', 'rsfmri_cor_ngd_vta_scs_vtdcrh']

visual_to_sub = ['rsfmri_cor_ngd_vs_scs_crcxlh', 'rsfmri_cor_ngd_vs_scs_thplh', 'rsfmri_cor_ngd_vs_scs_cdelh', 'rsfmri_cor_ngd_vs_scs_ptlh', 'rsfmri_cor_ngd_vs_scs_pllh', 'rsfmri_cor_ngd_vs_scs_bs', 'rsfmri_cor_ngd_vs_scs_hplh', 'rsfmri_cor_ngd_vs_scs_aglh', 'rsfmri_cor_ngd_vs_scs_aalh', 'rsfmri_cor_ngd_vs_scs_vtdclh', 'rsfmri_cor_ngd_vs_scs_crcxrh', 'rsfmri_cor_ngd_vs_scs_thprh', 'rsfmri_cor_ngd_vs_scs_cderh', 'rsfmri_cor_ngd_vs_scs_ptrh', 'rsfmri_cor_ngd_vs_scs_plrh', 'rsfmri_cor_ngd_vs_scs_hprh', 'rsfmri_cor_ngd_vs_scs_agrh', 'rsfmri_cor_ngd_vs_scs_aarh', 'rsfmri_cor_ngd_vs_scs_vtdcrh']

In [ ]:
subcort = pd.read_csv('mrirscor02.csv')

In [ ]:
col = subcort.loc[: , aud_to_sub]
subcort['aud_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , cingoper_to_sub]
subcort['cingoper_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , cingpar_to_sub]
subcort['cingpar_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , default_to_sub]
subcort['default_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , DAN_to_sub]
subcort['DAN_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , frontopar_to_sub]
subcort['frontopar_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , other_to_sub]
subcort['other_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , retrotemp_to_sub]
subcort['retrotemp_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , senshand_to_sub]
subcort['senshand_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , sensmouth_to_sub]
subcort['sensmouth_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , salience_to_sub]
subcort['salience_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , VAN_to_sub]
subcort['VAN_to_sub'] = col.mean(axis=1)

col = subcort.loc[: , visual_to_sub]
subcort['visual_to_sub'] = col.mean(axis=1)

In [ ]:
subcort.set_index('subjectkey', inplace= True)

In [ ]:
subcort.eventname.unique()
subcortt1 = subcort.loc[subcort['eventname'] == 'baseline_year_1_arm_1']
subcortt2 = subcort.loc[subcort['eventname'] == '2_year_follow_up_y_arm_1']

In [ ]:
source = ["Auditory", "Cingulo-opercular", "Cingulo-parietal", "DefaultMode", "DorsalAttention", "Fronto-parietal", "Other", "RetrosplenialTemporal", "SensorimotorHand", "SensorimotorMouth", "Salience", "VentralAttention", "Visual"]
subcortdf = pd.DataFrame(source, columns=["source"])
subcortdf = subcortdf.assign(target = 'Subcortical')

In [ ]:
#This was created via permutation and put into a csv 
#essentially a list of source:targets with every combination of network
adjacencylist = pd.read_csv('adjacencylist.csv')
adjacencylist = adjacencylist.rename(columns={"Source": "source", "Target": "target"})

In [ ]:
#check relevant columns (of every combination of network strength)
validp.iloc[:,203:372]

In [ ]:
columnlist = validp.columns.values
relevantcolumns = columnlist[203:372]

## Timepoint 1 Network Creation

In [ ]:
##TIMEPOINT1 

counter = 0 
#set up area under curve lists 
areaclist = []
areadlist = []
areaelist = []
areabclist = []
subjectkeylist = []

#---------------------
#BEGINNING OF FOR LOOP ACROSS PARTICIPANTS
# for subjectkey, row in validp.iterrows():
for subjectkey, row in validp.iterrows():
    adjlist2= adjacencylist.copy()
    #set up the edge matrix -relevantcolumns is the t1 rsfmri correlations 
    edgeweights = validp.loc[subjectkey, relevantcolumns].values
    adjlist2['weight'] = edgeweights
    
    #add subcortical measures 
    a= subcortt1.loc[subcortt1.index == subjectkey, 'aud_to_sub'].item()
    b= subcortt1.loc[subcortt1.index == subjectkey, 'cingoper_to_sub'].item()
    c= subcortt1.loc[subcortt1.index == subjectkey, 'cingpar_to_sub'].item()
    d= subcortt1.loc[subcortt1.index == subjectkey, 'default_to_sub'].item()
    e= subcortt1.loc[subcortt1.index == subjectkey, 'DAN_to_sub'].item()
    f= subcortt1.loc[subcortt1.index == subjectkey, 'frontopar_to_sub'].item()
    g= subcortt1.loc[subcortt1.index == subjectkey, 'other_to_sub'].item()
    h= subcortt1.loc[subcortt1.index == subjectkey, 'retrotemp_to_sub'].item()
    i= subcortt1.loc[subcortt1.index == subjectkey, 'senshand_to_sub'].item()
    j= subcortt1.loc[subcortt1.index == subjectkey, 'sensmouth_to_sub'].item()
    k= subcortt1.loc[subcortt1.index == subjectkey, 'salience_to_sub'].item()
    l= subcortt1.loc[subcortt1.index == subjectkey, 'VAN_to_sub'].item()
    m= subcortt1.loc[subcortt1.index == subjectkey, 'visual_to_sub'].item()
    
    #Add/update weight measures to the subcortical df 
    subcortdf['weight'] = [a,b,c,d,e,f,g,h,i,j,k,l,m]
#     display(subcortdf)

    #Append Subcortical connections to the main edgelist
    adjlist2 = adjlist2.append(subcortdf, ignore_index=True)
#     display(adjlist2)
    
    #Create graph 
    G = nx.from_pandas_edgelist(adjlist2, edge_attr='weight')
#     print(nx.average_clustering(G, weight='weight'))
    
    #Make minimum spanning fully connected graph
    Gmin = nx.minimum_spanning_tree(G, ignore_nan=True)
#     print(nx.average_clustering(Gmin, weight='weight'))
    
    #---------------------
    #CODE TO VISUALIZE THE NETWORK (fully connected)
#     pos=nx.spring_layout(G)
#     nx.draw_networkx_nodes(G,pos)

#     edges = G.edges()
#     print(G.edges())
#     print('length of G.edges', len(edges))
#     weights = [G[u][v]['weight'] for u,v in edges]

#     nx.draw(G, pos, edges=edges, width=10*weights, opacity=10*weights)
#     #
#     nx.draw_networkx_labels(G,pos)#,font_size=20,font_family='sans-serif')
#     #
#     plt.axis('off')
#     plt.show()
    #---------------------
      
    #---------------------
#   #CODE TO VISUALIZE THE NETWORK (minimum spanning)
#     pos=nx.spring_layout(Gmin)
#     nx.draw_networkx_nodes(Gmin,pos)

#     edges = Gmin.edges()
#     weights = [Gmin[u][v]['weight'] for u,v in edges]

#     nx.draw(Gmin, pos, edges=edges, width=100*weights, opacity=20*weights)
#     #
#     nx.draw_networkx_labels(G,pos)#,font_size=20,font_family='sans-serif')
#     #
#     plt.axis('off')
#     plt.show()
    #---------------------
    ##CODE FOR SPARSITY 
    
    #sort edges by weight and catch any missing data
    sortededges= adjlist2.sort_values(by='weight', ascending=False)
    if sum(sortededges.weight.isna()) > 0:
        print('skipped participant due to nans')
        continue
    
    #for loop for different sparsity assumptions
    sparselist = np.linspace(0.01,.4, 40)
    c=[]
    d=[]
    e=[]
    bc=[]
    for elem in sparselist: 
        num_edges = round(elem*len(sortededges)).astype(np.int)
        Gmin.add_weighted_edges_from(list(sortededges[0:num_edges].itertuples(index=False, name=None)))
        c.append(nx.average_clustering(Gmin))
        d.append(max(Gmin.degree)[1])
        e.append(nx.global_efficiency(Gmin))
        bc.append(max(nx.betweenness_centrality(Gmin).values()))
        

    # Compute the area using the composite trapezoidal rule.
    areac = trapz(c, dx=.01)
    aread = trapz(d, dx=.01)
    areae = trapz(e, dx=.01)
    areabc = trapz(bc, dx=.01)
#     print(y)
#     print("area =", area)
    
    #----------------------

    #---------------------    
    #FILLED LINE GRAPH OF AUC - clustering 
#     x=sparselist
#     # Area plot
#     plt.fill_between(x, y)
#     plt.title('Clustering coefficient under differing sparsity constraints')
#     plt.ylabel('clustering coefficient')
#     plt.xlabel('sparsity')
#     plt.show()

    #---------------------
    #Draw Gmin for .40 sparsity
#     from itertools import chain

#     colors = list(chain.from_iterable(d.values() for *_, d in Gmin.edges(data=True)))
#     pos=nx.spring_layout(Gmin)cc
#     nx.draw(Gmin,pos=pos, edge_cmap=plt.get_cmap('Blues'), edge_color = colors, node_color="#A0CBE2", with_labels=True, font_color='black', opacity = 20*np.abs(colors), width=1 +np.abs(colors))
#     plt.show()
#     ---------------------

    #append area values to lists
    subjectkeylist.append(subjectkey)
    areaclist.append(areac)
    areadlist.append(aread)
    areaelist.append(areae)
    areabclist.append(areabc)
    #This is a crude way of capturing participants and visualising participants who have NaNs in their networks 
#     if areac > 0.225:
#         colors = list(chain.from_iterable(d.values() for *_, d in Gmin.edges(data=True)))
#         nx.draw(Gmin,pos=pos, edge_cmap=plt.get_cmap('Blues'), edge_color = colors, node_color="#A0CBE2", with_labels=True, font_color='black', opacity = 20*np.abs(colors), width=1 +np.abs(colors))
#         plt.show()
#         print(subjectkey)
#         display(sortededges)
#         display(adjlist2)
    
    #---------------------
    #LIMIT/REPORT QUANTITY
    counter = counter + 1
#     Debug mode
#     if counter ==1500:
#         break
    if counter % 500 == 0:
        print(counter,' analyzed')

auct1 = pd.DataFrame(
    {'subjectkey':subjectkeylist,
     'clusteringt1': areaclist,
     'maxdegreet1': areadlist,
     'efficiencyt1': areaelist,
     'betweencentralityt1' : areabclist
    })
auct1

## Timepoint 2 Network Creation

In [ ]:
#Check columns for timepoint2
validp.iloc[:,395:564]

In [ ]:
relevantcolumns = columnlist[395:564]
validp[relevantcolumns]

In [ ]:
##TIMEPOINT 2

counter = 0 
#set up area under curve lists 
areaclist = []
areadlist = []
areaelist = []
areabclist = []
subjectkeylist = []

#---------------------
#BEGINNING OF FOR LOOP ACROSS PARTICIPANTS
for subjectkey, row in validp.iterrows():
    adjlist2= adjacencylist.copy()
    #set up the edge matrix -relevantcolumns is the t2 rsfmri correlations
    relevantcolumns = columnlist[395:564]
    edgeweights = validp.loc[subjectkey, relevantcolumns].values
    adjlist2['weight'] = edgeweights
    
    #add subcortical measures 
    a= subcortt2.loc[subcortt2.index == subjectkey, 'aud_to_sub'].item()
    b= subcortt2.loc[subcortt2.index == subjectkey, 'cingoper_to_sub'].item()
    c= subcortt2.loc[subcortt2.index == subjectkey, 'cingpar_to_sub'].item()
    d= subcortt2.loc[subcortt2.index == subjectkey, 'default_to_sub'].item()
    e= subcortt2.loc[subcortt2.index == subjectkey, 'DAN_to_sub'].item()
    f= subcortt2.loc[subcortt2.index == subjectkey, 'frontopar_to_sub'].item()
    g= subcortt2.loc[subcortt2.index == subjectkey, 'other_to_sub'].item()
    h= subcortt2.loc[subcortt2.index == subjectkey, 'retrotemp_to_sub'].item()
    i= subcortt2.loc[subcortt2.index == subjectkey, 'senshand_to_sub'].item()
    j= subcortt2.loc[subcortt2.index == subjectkey, 'sensmouth_to_sub'].item()
    k= subcortt2.loc[subcortt2.index == subjectkey, 'salience_to_sub'].item()
    l= subcortt2.loc[subcortt2.index == subjectkey, 'VAN_to_sub'].item()
    m= subcortt2.loc[subcortt2.index == subjectkey, 'visual_to_sub'].item()
    
    #Add/update weight measures to the subcortical df 
    subcortdf['weight'] = [a,b,c,d,e,f,g,h,i,j,k,l,m]
#     display(subcortdf)

    #Append Subcortical connections to the main edgelist
    adjlist2 = adjlist2.append(subcortdf, ignore_index=True)
#     display(adjlist2)
    
    #Create graph 
    G = nx.from_pandas_edgelist(adjlist2, edge_attr='weight')
#     print(nx.average_clustering(G, weight='weight'))
    
    #Make minimum spanning fully connected graph
    Gmin = nx.minimum_spanning_tree(G, ignore_nan=True)
#     print(nx.average_clustering(Gmin, weight='weight'))
    
    #---------------------
    #CODE TO VISUALIZE THE NETWORK (fully connected)
#     pos=nx.spring_layout(G)
#     nx.draw_networkx_nodes(G,pos)

#     edges = G.edges()
#     print(G.edges())
#     print('length of G.edges', len(edges))
#     weights = [G[u][v]['weight'] for u,v in edges]

#     nx.draw(G, pos, edges=edges, width=10*weights, opacity=10*weights)
#     #
#     nx.draw_networkx_labels(G,pos)#,font_size=20,font_family='sans-serif')
#     #
#     plt.axis('off')
#     plt.show()
    #---------------------
      
    #---------------------
#   #CODE TO VISUALIZE THE NETWORK (minimum spanning)
#     pos=nx.spring_layout(Gmin)
#     nx.draw_networkx_nodes(Gmin,pos)

#     edges = Gmin.edges()
#     weights = [Gmin[u][v]['weight'] for u,v in edges]

#     nx.draw(Gmin, pos, edges=edges, width=100*weights, opacity=20*weights)
#     #
#     nx.draw_networkx_labels(G,pos)#,font_size=20,font_family='sans-serif')
#     #
#     plt.axis('off')
#     plt.show()
    #---------------------
    ##CODE FOR SPARSITY 
    
    #sort edges by weight and catch nans
    sortededges= adjlist2.sort_values(by='weight', ascending=False)
    if sum(sortededges.weight.isna()) > 0:
        print('skipped participant due to nans')
        continue
    
    #for loop for different sparsity assumptions
    sparselist = np.linspace(0.01,.4, 40)
    c=[]
    d=[]
    e=[]
    bc=[]
    for elem in sparselist: 
        num_edges = round(elem*len(sortededges)).astype(np.int)
        Gmin.add_weighted_edges_from(list(sortededges[0:num_edges].itertuples(index=False, name=None)))
        c.append(nx.average_clustering(Gmin))
        d.append(max(Gmin.degree)[1])
        e.append(nx.global_efficiency(Gmin))
        bc.append(max(nx.betweenness_centrality(Gmin).values()))
        

    # Compute the area using the composite trapezoidal rule.
    areac = trapz(c, dx=.01)
    aread = trapz(d, dx=.01)
    areae = trapz(e, dx=.01)
    areabc = trapz(bc, dx=.01)
    
    subjectkeylist.append(subjectkey)
    areaclist.append(areac)
    areadlist.append(aread)
    areaelist.append(areae)
    areabclist.append(areabc)
#     print(y)
#     print("area =", area)
    
    #----------------------

    #---------------------    
#     FILLED LINE GRAPH OF AUC 
#     x=sparselist
#     # Area plot
#     plt.fill_between(x, y)
#     plt.title('Clustering coefficient under differing sparsity constraints')
#     plt.ylabel('clustering coefficient')
#     plt.xlabel('sparsity')
#     plt.show()
    #---------------------
#     Draw Gmin for .40 sparsity
#     from itertools import chain

#     colors = list(chain.from_iterable(d.values() for *_, d in Gmin.edges(data=True)))
#     pos=nx.spring_layout(Gmin)
#     nx.draw(Gmin,pos=pos, edge_cmap=plt.get_cmap('Blues'), edge_color = colors, node_color="#A0CBE2", with_labels=True, font_color='black', opacity = 20*np.abs(colors), width=1 +np.abs(colors))
#     plt.show()
#     #---------------------
    
    #---------------------
    #LIMIT/REPORT QUANTITY
    counter = counter + 1
#     if counter ==8:
#         break
    if counter % 500 == 0:
        print(counter,' analyzed')
        time.sleep(1)

auct2 = pd.DataFrame(
    {'subjectkey': subjectkeylist,
     'clusteringt2': areaclist,
     'maxdegreet2': areadlist,
     'efficiencyt2': areaelist,
     'betweencentralityt2' : areabclist
    })
auct2

## Create Network Area Under Curve Dataframe

In [ ]:
auct1 = auct1.set_index('subjectkey')
auct2 = auct2.set_index('subjectkey')

In [ ]:
mergetest = auct1.copy()
print(len(mergetest))
mergetest = mergetest.merge(auct2,left_index=True, right_index=True, how='inner')

In [ ]:
mergetest['clustdiff'] = mergetest['clusteringt2'] - mergetest['clusteringt1']
mergetest['degdiff'] = mergetest['maxdegreet2'] - mergetest['maxdegreet1']
mergetest['effdiff'] = mergetest['efficiencyt2'] - mergetest['efficiencyt1']
mergetest['bcdiff'] = mergetest['betweencentralityt2'] - mergetest['betweencentralityt1']

In [ ]:
finaldf = validp.merge(mergetest,left_index=True, right_index=True, how='inner')

## Calculate Change Over Time

In [ ]:
finaldf['lnclustdiff'] = np.log(finaldf['clusteringt2']) - np.log(finaldf['clusteringt1'])

In [ ]:
finaldf['lndegdiff'] = np.log(finaldf['maxdegreet2']) - np.log(finaldf['maxdegreet1'])

In [ ]:
finaldf['lneffdiff'] = np.log(finaldf['efficiencyt2']) - np.log(finaldf['efficiencyt1'])

In [ ]:
finaldf['lneffdiff'] = np.log(finaldf['efficiencyt2']) - np.log(finaldf['efficiencyt1'])
finaldf['lnbcdiff'] = np.log(finaldf['betweencentralityt2']) - np.log(finaldf['betweencentralityt1'])

## Model Building

In [ ]:
finaldf['agezscore'] = stats.zscore(finaldf['interview_age.t0'])

In [ ]:
finaldf['ST0'] = finaldf['ST.t0']
finaldf['STsqrt'] = np.sqrt(finaldf['ST0'])
finaldf['STsqrtz'] = stats.zscore(finaldf['STsqrt'])

In [ ]:
finaldf = finaldf.merge(demodf, left_index=True, right_index=True, how='left')

In [ ]:
#SMF formulas don't like periods in the names of variables so we need to rename 
finaldf.rename(columns={'sex.t0':'sex0'}, inplace=True)

In [ ]:
#recode to fit with smf 
finaldf['inc1'] = finaldf['<$25k']
finaldf['inc4'] = finaldf['>$100k']
finaldf['inc2']= finaldf['$25-50k']
finaldf['inc3'] = finaldf['$50-100k']

In [ ]:
#drop the duplicate 'sex' columns
finaldf = finaldf.loc[:,~finaldf.columns.duplicated()]

In [ ]:
#standardize to z-scores to output standardized coefficients
columns_to_standardize = ['lnclustdiff', 'lnbcdiff', 'lndegdiff', 'lneffdiff']
for column in columns_to_standardize: 
    titlecol = column+'z'
    finaldf[titlecol] = stats.zscore(finaldf[column])

## Time point comparisons - do these networks become more modular over time?

### Plots of AUC statistics

In [ ]:
matplotlib.style.use('fivethirtyeight')


In [ ]:
fig, ax = plt.subplots()
ax.xlim=(0.1, 0.25)
ax.ylim=(0.1, 0.25)
ax.set_xticks([.1, .12, .14, .16, .18, .20, .22, .24])
ax.set_yticks([.1, .12, .14, .16, .18, .20, .22, .24])
sns.regplot(x= 'betweencentralityt1', y= 'betweencentralityt2', scatter_kws={"color": "#f16546", 'alpha':0.10}, line_kws={"color": "k", "alpha":0.4, 'lw': 3}, data=finaldf)
plt.xlabel('AUC Between Centrality at Timepoint 1', fontsize=12)
plt.xticks(fontsize=10)
plt.ylabel('AUC Between Centrality at Timepoint 2', fontsize=12)
plt.yticks(fontsize=10)
plt.axis('square')
ax.xaxis.label.set_size(12)
plt.tight_layout()
plt.savefig("BCscatter.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.regplot(x= 'maxdegreet1', y= 'maxdegreet2', scatter_kws={"color": "#f16546", 'alpha':0.10}, line_kws={"color": "k", "alpha":0.4, 'lw': 3}, data=finaldf)
ax.set_xticks(range(1,6))
ax.set_yticks(range(1,6))
plt.xlabel('AUC Max Degree at Timepoint 1', fontsize=12)
plt.xticks(fontsize=10)
plt.ylabel('AUC Max Degree at Timepoint 2', fontsize=12)
plt.yticks(fontsize=10)
plt.axis('square')
plt.tight_layout()
plt.savefig("maxdegreescatter.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.regplot(x= 'efficiencyt1', y= 'efficiencyt2', scatter_kws={"color": "#f16546", 'alpha':0.10}, line_kws={"color": "k", "alpha":0.4, 'lw': 3}, data=finaldf)
ax.set_xticks([.22, .23, .24, .25])
ax.set_yticks([.22, .23, .24, .25])
plt.xlabel('AUC Efficiency at Timepoint 1', fontsize=12)
plt.xticks(fontsize=10)
plt.ylabel('AUC Efficiency at Timepoint 2', fontsize=12)
plt.yticks(fontsize=10)
plt.axis('square')
plt.tight_layout()
plt.savefig("efficiencyscatter.pdf")
plt.show()

In [ ]:
fig, ax = plt.subplots()
sns.regplot(x= 'clusteringt1', y= 'clusteringt2', scatter_kws={"color": "#f16546", 'alpha':0.15}, line_kws={"color": "k", "alpha":0.4, 'lw': 3}, data=finaldf)
ax.set_xticks([.1, .125, .15, .175, .2, .225])
ax.set_yticks([.1, .125, .15, .175, .2, .225])
plt.xlabel('AUC Clustering at Timepoint 1', fontsize=12)
plt.xticks(fontsize=10)
plt.ylabel('AUC Clustering at Timepoint 2', fontsize=12)
plt.yticks(fontsize=10)
plt.axis('square')
plt.tight_layout()
plt.savefig("clusterscatter.pdf")
plt.show()

## Perform multiple linear regressions

In [ ]:
#4 respondents did not give an answer to the Latinx ethnicity question
print(finaldf.Latinx.isnull().sum())
finaldf.Latinx = finaldf.Latinx.fillna(value=0)

### Clustering

In [ ]:
formula = 'lnclustdiffz ~ STsqrtz + C(White) +C(Latinx) +C(Black) + C(Asian) + agezscore + C(sex) + +C(inc1) + C(inc2) +C(inc3) +C(inc4)'
model = smf.ols(formula = formula,data=finaldf, missing='drop')
model = model.fit()
model.summary()

In [ ]:
model.conf_int(alpha=0.0125)

### Max Degree

In [ ]:
sns.regplot(data = finaldf, x = 'STsqrt', y='lndegdiff', scatter_kws={"color": "#ffca65", 'alpha':0.25}, line_kws={"color": "#f16546", "alpha":0.8, 'lw': 3}) 
plt.savefig('lnmaxdegvsst.pdf')
plt.show()

In [ ]:
formula = 'lndegdiffz ~ STsqrtz + C(White) +C(Latinx) +C(Black) + C(Asian) + agezscore + C(sex) + +C(inc1) + C(inc2) +C(inc3) +C(inc4)'
model = smf.ols(formula = formula,data=finaldf)
model = model.fit()
model.summary()

In [ ]:
model.conf_int(alpha=0.0125)

### Efficiency

In [ ]:
sns.regplot(data = finaldf, x = 'STsqrt', y='lneffdiff', scatter_kws={"color": "#ffca65", 'alpha':0.25}, line_kws={"color": "#f16546", "alpha":0.8, 'lw': 3}) 
plt.savefig('lneffdiffvsst.pdf')
plt.show()

In [ ]:
formula = 'lneffdiffz ~ STsqrtz + C(White) +C(Latinx) +C(Black) + C(Asian) + agezscore + C(sex) + +C(inc1) + C(inc2) +C(inc3) +C(inc4)'
model = smf.ols(formula = formula,data=finaldf)
model = model.fit()
model.summary()

In [ ]:
model.conf_int(alpha=0.0125)

### Between Centrality

In [ ]:
sns.regplot(data = finaldf, x = 'STsqrt', y='lnbcdiff', scatter_kws={"color": "#ffca65", 'alpha':0.25}, line_kws={"color": "#f16546", "alpha":0.8, 'lw': 3}) 
plt.savefig('lnbcdiffvsst.pdf')
plt.show()

In [ ]:
formula = 'lnbcdiffz ~ STsqrtz + C(White) +C(Latinx) +C(Black) + C(Asian) + agezscore + C(sex) + +C(inc1) + C(inc2) +C(inc3) +C(inc4)'
model = smf.ols(formula = formula,data=finaldf)
model = model.fit()
model.summary()

In [ ]:
model.conf_int(alpha=0.0125)

## Screen Time Correction Plots

In [ ]:
fig, ax = plt.subplots()
plt.hist(finaldf.ST0, bins=20, color ='#ffca65')
plt.xlabel('Weekly Hours of Screentime', fontsize=12)
plt.grid(color='lightgray')
plt.savefig("Screentimedistributionincl.pdf")

In [ ]:
fig, ax = plt.subplots()
plt.hist(finaldf.STsqrt, bins=20, color ='#ffca65')
plt.xlabel('Weekly Hours of Screentime', fontsize=12)
plt.grid(color='lightgray')
plt.savefig("sqrtScreentimedistributionincl.pdf")

## KDE plots

In [ ]:
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
sns.kdeplot(x=finaldf['STsqrtz'], y=finaldf['lnclustdiffz'], cmap="Reds", shade =True, bw_adjust=.5, cbar=True)
plt.grid(color='lightgray', linestyle='-', linewidth=2)
plt.tight_layout()
plt.savefig('clusterstkde.pdf')
plt.show()

In [ ]:
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
sns.kdeplot(x=finaldf['STsqrtz'], y=finaldf['lndegdiffz'], cmap="Reds", shade =True, bw_adjust=.5, cbar=True)
plt.grid(color='lightgray', linestyle='-', linewidth=2)
plt.tight_layout()
plt.savefig('maxdegkde.pdf')
plt.show()

In [ ]:
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
sns.kdeplot(x=finaldf['STsqrtz'], y=finaldf['lneffdiffz'], cmap="Reds", shade =True, bw_adjust=.5, cbar=True)
plt.grid(color='lightgray', linestyle='-', linewidth=2)
plt.tight_layout()
plt.savefig('effkde.pdf')
plt.show()

In [ ]:
sns.set(rc={'axes.facecolor':'white', 'figure.facecolor':'white'})
sns.kdeplot(x=finaldf['STsqrtz'], y=finaldf['lnbcdiffz'], cmap="Reds", shade =True, bw_adjust=.5, cbar=True)
plt.grid(color='lightgray', linestyle='-', linewidth=2)
plt.tight_layout()
plt.savefig('bckde.pdf')
plt.show()

## Network Figure Production

In [ ]:
labeldict = {'Auditory': 'Aud', 'Cingulo-opercular': 'Cing-oper',
 'Cingulo-parietal': 'Cing-par', 'DefaultMode': 'DMN',
 'DorsalAttention': 'DAN', 'Fronto-parietal': 'Fronto-par',
 'Other' : 'Other', 'RetrosplenialTemporal': 'Retrospl-Temp',
 'SensorimotorHand':'SRM Hand', 'SensorimotorMouth': 'SRM Mouth',
 'Salience': 'Salience', 'VentralAttention': 'VAN',
 'Visual': 'Visual', 'Subcortical': 'Subcort'
}

In [ ]:
pos=nx.spring_layout(Gmin)
from itertools import chain
example = ['NDAR_INV04EUBGTM']

##--------SET UP THE SMALLEST SPANNING NET
for subjectkey in example:
    adjlist2= adjacencylist.copy()
    #set up the edge matrix -relevantcolumns is the t2 rsfmri correlations 
    edgeweights = validp.loc[subjectkey, relevantcolumns].values
    adjlist2['weight'] = edgeweights
    
    #add subcortical measures 
    a= subcortt2.loc[subcortt2.index == subjectkey, 'aud_to_sub'].item()
    b= subcortt2.loc[subcortt2.index == subjectkey, 'cingoper_to_sub'].item()
    c= subcortt2.loc[subcortt2.index == subjectkey, 'cingpar_to_sub'].item()
    d= subcortt2.loc[subcortt2.index == subjectkey, 'default_to_sub'].item()
    e= subcortt2.loc[subcortt2.index == subjectkey, 'DAN_to_sub'].item()
    f= subcortt2.loc[subcortt2.index == subjectkey, 'frontopar_to_sub'].item()
    g= subcortt2.loc[subcortt2.index == subjectkey, 'other_to_sub'].item()
    h= subcortt2.loc[subcortt2.index == subjectkey, 'retrotemp_to_sub'].item()
    i= subcortt2.loc[subcortt2.index == subjectkey, 'senshand_to_sub'].item()
    j= subcortt2.loc[subcortt2.index == subjectkey, 'sensmouth_to_sub'].item()
    k= subcortt2.loc[subcortt2.index == subjectkey, 'salience_to_sub'].item()
    l= subcortt2.loc[subcortt2.index == subjectkey, 'VAN_to_sub'].item()
    m= subcortt2.loc[subcortt2.index == subjectkey, 'visual_to_sub'].item()
    
    #Add/update weight measures to the subcortical df 
    subcortdf['weight'] = [a,b,c,d,e,f,g,h,i,j,k,l,m]
#     display(subcortdf)

    #Append Subcortical connections to the main edgelist
    adjlist2 = adjlist2.append(subcortdf, ignore_index=True)
#     display(adjlist2)
    
    #Create graph 
    G = nx.from_pandas_edgelist(adjlist2, edge_attr='weight')
#     print(nx.average_clustering(G, weight='weight'))
    
    #Make minimum spanning fully connected graph
    Gmin = nx.minimum_spanning_tree(G, ignore_nan=True)
#     print(nx.average_clustering(Gmin, weight='weight'))

#-------------------
    sparselist = np.linspace(0.01,.4, 40)
    y=[]
    counter = 0
    for elem in sparselist: 
        counter = counter + 1
        num_edges = round(elem*len(sortededges)).astype(np.int)
        Gmin.add_weighted_edges_from(list(sortededges[0:num_edges].itertuples(index=False, name=None)))
        y.append(nx.average_clustering(Gmin))
        colors = list(chain.from_iterable(d.values() for *_, d in Gmin.edges(data=True)))
        plt.figure(figsize=[7,5])
        plt.margins(x=0.4)
        nx.draw(Gmin,pos=pos, labels = labeldict, edge_cmap=plt.get_cmap('rocket_r'), edge_color = colors, node_color="#ffca65", with_labels=True, font_color='black', opacity = 1, width=2 +np.abs(colors))
        plotname = 'network'+str(counter)+'.pdf'
        plt.savefig(plotname)
        plt.show()

In [ ]:
#     FILLED LINE GRAPH OF AUC 
x=sparselist
# Area plot
plt.fill_between(x, y, color='#ffca65')
plt.title('Clustering coefficient under differing sparsity constraints', fontsize=12)
plt.ylabel('clustering coefficient', fontsize=12)
plt.xlabel('sparsity', fontsize=12)
plt.tight_layout()
plt.savefig('clusteringexample.pdf')
plt.show()